# This Notebook Simulates a Two Robots Orbiting Around a Each Other

## Required Imports

In [18]:
%load_ext autoreload
%autoreload 2

import math
import numpy as np

from waltz import Fish

from interaction import Interaction
from environment import Environment
from channel import Channel
from observer import Observer
from dynamics import Dynamics

from utils import generate_distortion, generate_fish, run_simulation

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Simulation of Waltzing

Change parameters here, e.g. the initializations of the fishes.

In [19]:
run_time = 90 # in seconds
num_fish = 2
arena_size = np.array([1780, 1780, 1170])
arena_center = arena_size / 2.0
fish_pos = np.transpose(np.array([[1780/2+300, 1780/2-500], [1780/2-200, 1780/2], [1170/2-50, 1170/2+100]]))
fish_vel = np.zeros((num_fish, 3))
fish_phi = np.array([[0.4 * math.pi], [1.6 * math.pi]])
fish_vphi = np.zeros((num_fish, 1))
clock_freqs = 1
verbose = False

distortion = generate_distortion(type='none', magnitude=130, n=math.ceil(arena_size[0]/10)+1, show=False)
environment = Environment(
    arena_size=arena_size,
    node_pos=fish_pos,
    node_vel=fish_vel,
    node_phi=fish_phi,
    node_vphi=fish_vphi,
    distortion=distortion,
    prob_type='binary',
    conn_thres=3000,
    conn_drop=1,
    noise_magnitude=10,
    verbose=verbose
)
interaction = Interaction(environment, verbose=verbose)
channel = Channel(environment)
dynamics = Dynamics(environment, clock_freq=clock_freqs)

fish = generate_fish(
    n=num_fish, 
    channel=channel, 
    interaction=interaction,
    dynamics=dynamics,
    target_dist=130*1.75,
    lim_neighbors=[2,3],
    neighbor_weights=1.0,
    fish_max_speeds=130,
    clock_freqs=clock_freqs,
    verbose=verbose
)
channel.set_nodes(fish)

observer = Observer(fish=fish, environment=environment, channel=channel)
    
run_simulation(fish=fish, observer=observer, run_time=run_time, dark=False, white_axis=False, no_legend=True)

Please wait patiently 90 seconds. Thanks.
It's time to say bye bye!


## Prepare Data for Animation

In [20]:
# Get fish data from observer
data = np.zeros((6, run_time, num_fish))
data[0, :, :] = np.transpose(np.array(observer.x))
data[1, :, :] = np.transpose(np.array(observer.y))
data[2, :, :] = np.transpose(np.array(observer.z))
data[3, :, :] = np.transpose(np.array(observer.vx))
data[4, :, :] = np.transpose(np.array(observer.vy))
data[5, :, :] = np.transpose(np.array(observer.vz))

## Save Data

In [21]:
np.save('waltz', data)

## Animate Simulated Data

Change the speed-up factor of the animation in the variable `speed`. 

In [8]:
# Imports
import ipyvolume as ipv
from ipyvolume.moviemaker import MovieMaker
import ipywidgets as widgets
import ipywebrtc as webrtc
import matplotlib.cm as cm
import time

# Data handling
x, y, z, vx, vy, vz = data
speed = 10 # speed up animation 10 times

# Colors
colors[:, -2:, :] = cm.Reds(0.5) # one robot is red
colors[:, -1:, :] = cm.Blues(0.5) # one robot is blue

# Figure
fig = ipv.figure()
ipv.xlim(0, 1780)
ipv.ylim(0, 1780)
ipv.zlim(0, 1170)
ipv.style.use('dark')
quiver = ipv.quiver(x, y, z, vx, vy, vz, size=10, color=colors[:,:,:3])
ipv.animation_control(quiver, interval=1000/speed)
ipv.show()

ModuleNotFoundError: No module named 'ipyvolume'

## Save Animation as html

In [ ]:
ipv.save('waltz.html')
!open 'waltz.html'

# Visualize Trace
This animation can also show the trace of two waltzing fish. Exectue the following cells of this section only if you wish to see that trace.

In [5]:
dur_waltz = 40

data_trace0 = data[:, :dur_waltz, 0]
data_trace0 = data_trace0.reshape(6, 1, dur_waltz)
x0, y0, z0, vx0, vy0, vz0 = data_trace0

data_trace1 = data[:, :dur_waltz, 1]
data_trace1 = data_trace1.reshape(6, 1, dur_waltz)
x1, y1, z1, vx1, vy1, vz1 = data_trace1

v0 = vx0**2 + vy0**2 + vz0**2
v0 -= v0.min(); v0 /= v0.max();
colors0 = np.array([cm.Reds(k) for k in v0])

v1 = vx1**2 + vy1**2 + vz1**2
v1 -= v1.min(); v1 /= v1.max();
colors1 = np.array([cm.Blues(k) for k in v1])

# Figure
fig = ipv.figure()
ipv.xlim(0, 1780)
ipv.ylim(0, 1780)
ipv.zlim(0, 1170)
ipv.style.use('dark')
quiver = ipv.quiver(x0, y0, z0, vx0, vy0, vz0, size=10, color=colors0[:,:,:3])
quiver = ipv.quiver(x1, y1, z1, vx1, vy1, vz1, size=10, color=colors1[:,:,:3])
ipv.animation_control(quiver, interval=1000)
ipv.show()